In [1]:
missing_entries = 0

def generate_label(open_price, close_price) -> int:
    global missing_entries
    movement_percent = (close_price - open_price) / open_price * 100
    if movement_percent <= -0.5:
        return 0
    elif movement_percent >= 0.55:
        return 1
    else:
        missing_entries += 1
        return -1

import pandas as pd

df = pd.read_csv('dataset/price/raw/ABB.csv')
path = 'dataset/price/raw/ABB.csv'

stock_name = path.split('/')[-1].split('.')[0]

df['Name'] = stock_name
df['Label'] = df.apply(lambda row: generate_label(row['Open'], row['Close']), axis=1)
print(len(df))

df = df[~(df['Label'] == -1)]

print(len(df))

df['Date'] = pd.to_datetime(df['Date'])

# Define start and end dates
start_date = '2014-01-01'
end_date = '2015-08-01'

# Filter the DataFrame for dates between start_date and end_date
filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

print(filtered_df)

1258
520
          Date       Open       High        Low      Close  Adj Close  \
333 2014-01-02  26.170000  26.170000  25.780001  25.930000  22.476974   
336 2014-01-07  25.709999  25.910000  25.680000  25.870001  22.424965   
339 2014-01-10  26.100000  26.299999  26.070000  26.299999  22.797699   
340 2014-01-13  26.230000  26.299999  26.040001  26.080000  22.607000   
341 2014-01-14  26.260000  26.430000  26.160000  26.420000  22.901722   
..         ...        ...        ...        ...        ...        ...   
725 2015-07-24  20.360001  20.379999  19.969999  20.000000  18.578381   
726 2015-07-27  19.889999  19.889999  19.639999  19.680000  18.281128   
727 2015-07-28  19.709999  19.900000  19.670000  19.870001  18.457623   
728 2015-07-29  19.809999  20.059999  19.799999  19.940001  18.522646   
730 2015-07-31  20.389999  20.410000  20.250000  20.280001  18.838480   

      Volume Name  Label  
333  2720400  ABB      0  
336   896200  ABB      1  
339  1297500  ABB      1  
340  1

In [7]:
import pandas as pd

def get_train_data(path:str) -> pd.DataFrame:
    df = pd.read_csv(path)

    stock_name = path.split('.')[0]
    if '/' in stock_name:
        stock_name = stock_name.split('/')[-1]

    df['Name'] = stock_name

    df['Label'] = df.apply(lambda row: generate_label(row['Open'], row['Close']), axis=1)

    df = df[~(df['Label'] == -1)]

    df['Date'] = pd.to_datetime(df['Date'])

    # Define start and end dates
    start_date = '2014-01-01'
    end_date = '2015-08-02'

    # Filter the DataFrame for dates between start_date and end_date
    filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

    return filtered_df



In [8]:
import os
missing_entries = 0
root = 'dataset/price/raw'
for idx, stock_path in enumerate(os.listdir(root)):
    df_stock = get_train_data(os.path.join(root, stock_path))
    if idx == 0:
        df_train = df_stock
    else:
        df_train = pd.concat([df_train, df_stock])
    
print(len(df_train))
print(missing_entries)
print(df_train.head())
    

17856
52266
          Date       Open       High        Low      Close  Adj Close  \
333 2014-01-02  23.340000  23.340000  23.030001  23.129999  19.080498   
334 2014-01-03  23.230000  23.330000  23.030001  23.110001  19.064001   
335 2014-01-06  23.059999  23.219999  22.850000  22.920000  18.907265   
338 2014-01-09  22.910000  23.209999  22.129999  22.350000  18.437057   
340 2014-01-13  22.080000  22.350000  22.080000  22.260000  18.362814   

       Volume  Name  Label  
333  294200.0  BSAC      0  
334  204800.0  BSAC      0  
335  257200.0  BSAC      0  
338  422500.0  BSAC      0  
340  286100.0  BSAC      1  


In [9]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Name', 'Date']).reset_index(drop=True)

lag = 5

lagged_columns = []
def create_lagged_features(group):
    for i in range(lag, 0, -1):
        group[f'Open_t_minus_{i}'] = group['Open'].shift(i)
        group[f'Close_t_minus_{i}'] = group['Close'].shift(i)
        # Append only if columns are new to avoid duplicates
        if f'Open_t_minus_{i}' not in lagged_columns:
            lagged_columns.append(f'Open_t_minus_{i}')
        if f'Close_t_minus_{i}' not in lagged_columns:
            lagged_columns.append(f'Close_t_minus_{i}')
    return group

# Apply the function to each group
df = df.groupby('Name').apply(create_lagged_features)

# Drop rows with NaN values in any of the lagged columns
df = df.dropna(subset=lagged_columns).reset_index(drop=True)

print(df.head())

        Date       Open       High        Low      Close  Adj Close   Volume  \
0 2012-09-13  19.209999  19.570000  19.110001  19.469999  16.341057  4038700   
1 2012-09-19  19.629999  19.860001  19.530001  19.780001  16.601238  1928400   
2 2012-09-20  19.370001  19.530001  19.309999  19.480000  16.349445  1553200   
3 2012-09-21  19.700001  19.709999  19.480000  19.500000  16.366238  1532400   
4 2012-09-25  19.379999  19.420000  19.030001  19.030001  15.971768  1467400   

  Name  Label  Open_t_minus_5  Close_t_minus_5  Open_t_minus_4  \
0  ABB      1       17.340000        17.240000       17.219999   
1  ABB      1       17.219999        17.320000       17.690001   
2  ABB      1       17.690001        17.910000       18.760000   
3  ABB      0       18.760000        18.540001       18.719999   
4  ABB      0       18.719999        18.900000       19.209999   

   Close_t_minus_4  Open_t_minus_3  Close_t_minus_3  Open_t_minus_2  \
0        17.320000       17.690001        17.910000

/tmp/ipykernel_10218/2783819110.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Name').apply(create_lagged_features)


In [14]:
df = df_train

lag = 5



In [21]:
from sklearn.preprocessing import LabelEncoder

def create_rnn_input(df):
    # Ensure 'Date' column is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Sort the DataFrame by 'Name' and 'Date'
    df = df.sort_values(['Name', 'Date']).reset_index(drop=True)

    # List to store the new rows
    data_rows = []

    name_encoder = LabelEncoder()
    df['Name'] = name_encoder.fit_transform(df['Name'])

    # Group the DataFrame by 'Name' (stock)
    grouped = df.groupby('Name')

    # For each stock group
    for name, group in grouped:
        group = group.sort_values('Date').reset_index(drop=True)

        # Skip if the group has less than 6 rows (need at least 5 days of history)
        if len(group) < 6:
            continue

        # Iterate over the group starting from the 5th index
        for idx in range(5, len(group)):
            # Get the previous 5 trading days
            prev_data = group.iloc[idx-5:idx]

            # Prepare the row data
            row_dict = {'Date': group.loc[idx, 'Date'], 'Name': name}

            # Add Open and Close prices from previous 5 trading days
            for i in range(5):
                row_dict[f'Open_t-{5-i}'] = prev_data.iloc[i]['Open']
                row_dict[f'Close_t-{5-i}'] = prev_data.iloc[i]['Close']

            # Add the Label for the target date
            row_dict['Label'] = group.loc[idx, 'Label']

            # Append the row to data_rows
            data_rows.append(row_dict)

    # Create a new DataFrame from data_rows
    new_df = pd.DataFrame(data_rows)

    x = new_df.drop(['Label', 'Date'], axis=1)
    y = new_df['Label']
    x = x.to_numpy()
    y = y.to_numpy()

    return x, y

new_df = create_rnn_input(df)
print(new_df.head())
print(len(df), len(new_df))

[ 0.       78.980003 77.28286  76.778572 77.704285 77.760002 77.148575
 76.972855 77.637146 78.114288 76.645714]
0
        Date  Name   Open_t-5  Close_t-5   Open_t-4  Close_t-4   Open_t-3  \
0 2014-01-10     0  78.980003  77.282860  76.778572  77.704285  77.760002   
1 2014-01-13     0  76.778572  77.704285  77.760002  77.148575  76.972855   
2 2014-01-14     0  77.760002  77.148575  76.972855  77.637146  78.114288   
3 2014-01-15     0  76.972855  77.637146  78.114288  76.645714  77.118568   
4 2014-01-17     0  78.114288  76.645714  77.118568  76.134285  75.701431   

   Close_t-3   Open_t-2  Close_t-2   Open_t-1  Close_t-1  Label  
0  77.148575  76.972855  77.637146  78.114288  76.645714      0  
1  77.637146  78.114288  76.645714  77.118568  76.134285      1  
2  76.645714  77.118568  76.134285  75.701431  76.532860      1  
3  76.134285  75.701431  76.532860  76.888573  78.055717      1  
4  76.532860  76.888573  78.055717  79.074287  79.622856      0  
17856 17421
